Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the related LOs requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>CS152 Assignment 2: The DPLL Algorithm</h1>

<h1>Question 1</h1>

Define your <code>Literal</code> class below.  Ensure that you define a <code>name</code> and <code>sign</code> attribute as required in the assignment description.  In addition, include any other attributes and member functions as needed. You will need to overload <code>__neg__()</code> to correctly handle negated literals.  You may need to overload <code>__eq__()</code> and <code>__hash__()</code> also.

In [1]:
# Import any packages you need here
# Also define any variables as needed

class Literal:
    def __init__(self, name, sign=True):
        self.name = name
        self.sign = sign
    
    def __neg__(self):
        return(Literal(self.name, not self.sign))
    
    def __eq__(self, other):
        return(self.name == other.name and self.sign == other.sign)
    
    def __abs__(self):
        if self.sign: return(self)
        else: return(-self)
        
    def __lt__(self,other):
        return(self.name < other.name)

    def __hash__(self):
        return(hash(str(self)))  
    
    def __repr__(self):
        if self.sign: return(self.name)
        else: return("-" + self.name)


<h1>Question 2</h1>
Implement DPLL by filling in the stubs below.  Note that the various heuristics are not required to be implemented for basic functionality, but a template has been provided for you to do so if you attempt the extension questions

In [2]:
# Define degree, pure symbol and unit clause heuristics here (optional).
# Add in your functions under the templates below
import copy
import collections

# _____ HEURISTICS _____
def first(clauses):
    return(abs(list(clauses[0])[0]))

def freq(clauses):
    ls = [abs(l) for c in clauses for l in c]
    return(max(sorted(ls)[::-1], key = ls.count))

def _getPureNUnit(clauses):
    ls = [l for c in clauses for l in c]
    pure = {l for l in ls if -l not in ls}
    unit = {l for c in clauses for l in c if len(c) == 1}
    return([abs(l) for l in pure] + [abs(l) for l in unit])

def pureNUnit(clauses):
    pNU = _getPureNUnit(clauses)
    if len(pNU) == 0: return(freq(clauses))
    return(sorted(pNU)[0])

def pureNUnitAlpha(clauses):
    ls = [abs(l) for c in clauses for l in c]
    pNU = _getPureNUnit(clauses)
    if len(pNU) == 0: return(freq(clauses))
    return(max(sorted(pNU), key = ls.count))
# _____________________


def DPLL_Satisfiable(KB, heuristic_level=0):
    ''' 
    satisfiable, model = DPLLSatisfiable(KB)
    Takes in a KB and returns whether the KB is satisfiable, and the model that makes it so
        
    KB: A knowledge base of clauses (CNF) consisting of a list of sets of literals.  A KB might look like
        [{A,B},{-A,C,D}]
    heuristic_level: An integer that will be passed in to specify which heuristics to implement 
        (only for the extension activities).
    satisfiable: Returns True if the KB is satisfiable, or False otherwise
    Model: A dictionary that assigns a truth value to each literal for the model that satisfies KB.
        For example, a model might look like {A: True, B: False}
    '''
    
    def _update(clauses,fact):
        '''
        Helper function to update clauses based on a new fact assignment
        '''
        updated = copy.deepcopy(clauses) # begone references!!
        updated = [c for c in updated if fact not in c]
        [c.discard(-fact) for c in updated]
        return(updated)
    
    hlookup = {
        0: first,
        1: freq,
        2: pureNUnit,
        3: pureNUnitAlpha
    }
    
    # global vars
    heuristic = hlookup[heuristic_level]
    trace = []
    
    def DPLL(clauses, model):
        if not clauses: # all clauses satisfied
            return(True, model, trace)
        elif set() in clauses: # an unsatisfiable clause exists
            return(False, model, trace)
        else:
            # pick a literal
            L = heuristic(clauses)  
            trace.append(L)
            
            # try positive
            model[L] = True
            pres, pmodel, ptrace = DPLL(_update(clauses,L),model)
            if pres: return(pres, pmodel, ptrace)
            
            # try negative
            model[L] = False
            nres, nmodel, ntrace = DPLL(_update(clauses,-L),model)
            return(nres, nmodel, ntrace)
    return(DPLL(KB,{}))


<h1>Question 3</h1>

Implement your KB from Russell & Norvig in CNF by filling in the sets inside the list <code>KB</code> below.  Ensure that your KB is in a list of set format as stated in the assignment instructions.

In [3]:
# Define some literals
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

KB = [{-A,B,E},{A,-B},{A,-E},{-E,D},{-C,-F,-B},{-E,B},{-B,F},{-B,C}]

<h1>Extensions</h1>

1. Implement the degree heuristic for choosing symbols.  If <code>heuristic_level=1</code>, then the degree heuristic should be used to select which symbol to assign.
2. Implement the pure symbol and unit clause heuristics.  If <code>heuristic_level=2</code>, then these heuristics should be used to find select first a pure symbol, and if one is not found, then a unit clause.  If neither pure symbols nor unit clauses are found, then the degree heuristic should be used to select a symbol.  If there are multiple pure symbols or unit clauses found, then use alphabetical order to select amongst them.
3. Modify the pure symbol heuristic to choose the pure symbol that occurs in the most number of clauses.  This should be activated if <code>heuristic_level=3</code>.

<h1>Basic Functionality Tests</h1>

All of the tests in this section must be passed for the code to be considered fully functional.  Other unseen test cases will be used after submission to further verify functionality

In [4]:
# This section will test the correct definition of the literal class


# Test the name attribute works correctly
assert(A.name == 'A')

# Test that negation works correctly
assert(not (-C).sign)

In [5]:
# This section will test that the KB has been correctly converted to CNF by performing 
# tests over all possible models
import itertools
symbols = {A,B,C,D,E,F}
symbol_list = [A,B,C,D,E,F]

def evaluate_russell_norvig_KB(model):
    # Manually evaluate the KB using the model
    s = list()
    s.append(model[A] == (model[B] or model[E]))
    s.append(model[E] <= model[D])
    s.append((model[C] and model[F]) <= (not model[B]))
    s.append(model[E] <= model[B])
    s.append(model[B] <= model[F])
    s.append(model[B] <= model[C])
    return all(s)

def evaluate_KB(KB, model):
    model_true = True
    model_name_dict = {l.name: model[l] for l in model}
    for clause in KB:
        evaluation = {l.sign == model_name_dict[l.name] for l in clause if l.name in model_name_dict}
        model_true = model_true and any(evaluation)
    return model_true

all_models = list(itertools.product([False, True], repeat=6))
for i in range(0, len(all_models)):
    # Test all truth values
    test_model = {symbol_list[j]: all_models[i][j] for j in range(0,6)}
    
    # Test the model
    assert(evaluate_russell_norvig_KB(test_model) == evaluate_KB(KB, test_model))

In [6]:
# This section will test the basic working of DPLL
# Satisfiable model
test_KB = [{A, C},{-A, C}, {B, -C}]
t, model, symbol_trace = DPLL_Satisfiable(test_KB)
assert(evaluate_KB(test_KB, model))
assert(t)

# Unsatisfiable model
test_KB = [{A, C},{-A, C}, {-C}]
t, model, symbol_trace = DPLL_Satisfiable(test_KB)
assert(not t)

In [7]:
# This will test DPLL on the KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,0)
print(model)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model)) 

{E: False, A: False, D: False, C: False, F: False, B: False}


<h1>Extension Tests</h1>

This section will test that the degree heuristic, pure symbol and unit clause heuristics are correctly implemented.  Note that in your code, the <code>heuristic_level</code> needs to correctly activate the heuristic being tested, and the <code>symbol_list</code> needs to be correctly generated 

In [8]:
# Degree Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,1)
print(model)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))  

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] == ['B','F', 'E', 'C', 'C', 'E', 'A'])

{B: False, F: False, E: False, C: False, A: False}


In [9]:
# Pure Symbol & Unit Clause Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,2)
print(model)


# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] == ['D', 'B', 'E', 'A', 'C', 'F', 'E', 'A'])

{D: True, B: False, A: False, C: False, F: False, E: False}


AssertionError: 

In [10]:
# Pure Symbol & Unit Clause Heuristic Test, choosing the most-frequently used pure symbol, for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,3)
print(model)
print(symbol_trace)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] == ['D', 'B', 'A', 'C', 'F', 'E', 'A'])

{D: True, B: False, A: False, C: False, F: False, E: False}
[D, B, A, C, F, E, A]
